<h1>Generate Mistrust Scores for All Patients</h1>
- Supervised Machine Learning (binary chartevents features) to predict whether "noncompliance" appears in the notes
- Supervised Machine Learning (binary chartevents features) to predict whether the patient or family consents/declines autopsy
- Off the shelf sentiment analysis of the notes

In [1]:
from collections import Counter
from collections import defaultdict
import cPickle as pickle
import numpy as np
import pandas as pd
import psycopg2
import random
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
import tqdm
from time import gmtime, strftime

In [2]:
con = psycopg2.connect(dbname ='mimic', user='wboag', host="/var/run/postgresql")
cur = con.cursor()

<h1>Load Data from MIMIC</h1>

In [3]:
# Read interpersonal interaction variables from chartevents

relevant_labels = '''
Family Communication
Follows Commands
Education Barrier
Education Learner
Education Method
Education Readiness
Education Topic #1
Education Topic #2
Pain
Pain Level
Pain Level (Rest)
Pain Assess Method
Restraint
Restraint Type
Restraint (Non-violent)
Restraint Ordered (Non-violent)
Restraint Location
Reason For Restraint
Spiritual Support
Support Systems
State
Behavior
Behavioral State
Reason For Restraint
Stress
Safety
Safety Measures_U_1
Family
Patient/Family Informed
Pt./Family Informed
Health Care Proxy
BATH                
bath                
Bath                
Bed Bath            
bed bath            
bed bath            
Bedbath             
CHG Bath            
Skin Care           
Judgement           
Family Meeting held 
Emotional / physical / sexual harm by partner or close relation
Verbal Response
Side Rails
Orientation
RSBI Deferred
Richmond-RAS Scale
Riker-SAS Scale
Status and Comfort
Teaching directed toward
Consults
Social work consult
Sitter
security
safety
headache
hairwashed
observer
'''

matches_text = []
for rl in relevant_labels.split('\n'):
    rl = rl.strip()
    if len(rl):
        matches_text.append( "LOWER(label) LIKE '%%%s%%'" % rl.lower())
matches = ' or '.join(matches_text)

print strftime("%Y-%m-%d %H:%M:%S", gmtime())
chartevents_query = 'select distinct hadm_id,label,value from mimiciii.chartevents c JOIN mimiciii.d_items i on i.itemid=c.itemid where (%s)' % matches
chartevents = pd.read_sql_query(chartevents_query, con)
print strftime("%Y-%m-%d %H:%M:%S", gmtime())

chartevents.head()

2018-07-13 11:04:16
2018-07-13 11:05:17


,hadm_id,label,value
0,100001,Baseline pain level,None
1,100001,Bed Bath,1
2,100001,CHG Bath,1
3,100001,Currently experiencing pain,0
4,100001,Education Barrier,None


In [4]:
# Read Notes
print strftime("%Y-%m-%d %H:%M:%S", gmtime()) 
notes_query = \
"""
select distinct n.hadm_id,n.category,n.text,n.chartdate,n.charttime
from mimiciii.noteevents n
where iserror IS NULL --this is null in mimic 1.4, rather than empty space
and hadm_id IS NOT NULL
;
"""
notes = pd.read_sql_query(notes_query, con)
print strftime("%Y-%m-%d %H:%M:%S", gmtime()) 

notes.head()

2018-07-13 11:05:17
2018-07-13 11:06:46


,hadm_id,category,text,chartdate,charttime
0,100001,Discharge summary,Admission Date: [**2117-9-11**] ...,2117-09-17,NaT
1,100001,Radiology,[**2117-9-11**] 11:12 AM\n CHEST (PA & LAT) ...,2117-09-11,2117-09-11 11:12:00
2,100003,Discharge summary,Admission Date: [**2150-4-17**] ...,2150-04-21,NaT
3,100003,ECG,Sinus rhythm\nProlonged QT interval is nonspec...,2150-04-17,NaT
4,100003,Echo,PATIENT/TEST INFORMATION:\nIndication: Left ve...,2150-04-18,NaT


<h1> Extract Features and Labels</h1>

In [5]:
# Extract features from chartevents

chartevents_features = {}
for hadm_id,rows in tqdm.tqdm(chartevents.groupby('hadm_id')):
    feats = {}
    for i,row in rows.iterrows():
        label = row.label.lower()
        
        if row.value is None:
            val = 'none'
        else:
            val = row.value.lower()
        
        
        if 'reason for restraint' in label:       
            
            if (val == 'not applicable') or (val == 'none'):
                val = 'none'
            elif ('threat' in val) or ('acute risk of' in val):
                val = 'threat of harm'
            elif ('confusion' in val) or ('delirium' in val) or (val == 'impaired judgment') or (val == 'sundowning'):
                val = 'confusion/delirium'
            elif ('occurence' in val) or (val == 'severe physical agitation') or (val == 'violent/self des'):
                val = 'prescence of violence'
            elif (val == 'ext/txinterfere') or (val == 'protection of lines and tubes') or (val == 'treatment interference'):
                val = 'treatment interference'
            elif 'risk for fall' in val:
                val = 'risk for falls'
            else:
                val = val
            
            feats[('reason for restraint', val)] = 1

        elif 'restraint location' in label:
                
            if val == 'none':
                val = 'none'
            elif '4 point rest' in val:
                val = '4 point restraint'
            else:
                val = 'some restraint'
            
            feats[('restraint location', val)] = 1

        elif 'restraint device' in label:
                
            if 'sitter' in val:
                val = 'sitter'
            elif 'limb' in val:
                val = 'limb'
            else:
                val = val
            
            feats[('restraint device', val)] = 1
            
        elif 'bath' in label:
            if 'part' in label:
                val = 'partial'
            elif 'self' in val:
                val = 'self'
            elif 'refused' in val:
                val = 'refused'
            elif 'shave' in val:
                val = 'shave'
            elif 'hair' in val:
                val = 'hair'
            elif 'none' in val:
                val = 'none'
            else:
                val = 'done'
            
            feats[('bath', val)] = 1

        elif label in ['behavior', 'behavioral state']:
            #feats[('behavior', val)] = 1
            pass
            
        elif label.startswith('pain level'):
            feats[('pain level', val)] = 1
            
        elif label.startswith('pain management'):
            #feats[('pain management', val)] = 1
            pass
        elif label.startswith('pain type'):
            #feats[('pain type', val)] = 1
            pass
        elif label.startswith('pain cause'):
            #feats[('pain cause', val)] = 1
            pass
        elif label.startswith('pain location'):
            #feats[('pain location', val)] = 1
            pass
            
        elif label.startswith('education topic'):
            feats[('education topic', val)] = 1

        elif label.startswith('safety measures'):
            feats[('safety measures', val)] = 1
            
        elif label.startswith('side rails'):
            feats[('side rails', val)] = 1
    
        elif label.startswith('status and comfort'):
            feats[('status and comfort', val)] = 1

        elif 'informed' in label:
            feats[('informed', val)] = 1        
        else:

            if type(row.value) == type(''):
                # extract phrase
                featname = (row.label.lower(), row.value.lower())
                value = 1.0
                feats[featname] = value
            elif row.value is None:
                featname = (row.label.lower(),'none')
                value = 1.0
                feats[featname] = value
            else: 
                featname = (row.label.lower(),)
                value = value
                feats[featname] = value
                pass
            
    
    chartevents_features[hadm_id] = feats


100%|██████████| 54510/54510 [04:45<00:00, 190.84it/s]


In [6]:
# LABEL: noncompliance in notes

mistrust_ids = []
for hadm_id,rows in tqdm.tqdm(notes.groupby('hadm_id')):
    # This is customizable to various note-based definitions of what to look for
    mistrust = False
    for text in rows.text.values:
        if 'noncompliant' in text.lower():
            mistrust = True
            
    # add the ID
    if mistrust:
        mistrust_ids.append(hadm_id)

        
# binary labels
trust_labels_noncompliance = {hadm_id:'trust' for hadm_id in chartevents['hadm_id'].values}
for hadm_id in mistrust_ids:
    trust_labels_noncompliance[int(hadm_id)] = 'mistrust'
    
print 'patients labeled as    trustful:', len([y for y in trust_labels_noncompliance.values() if y=='trust'   ])
print 'patients labeled as mistrustful:', len([y for y in trust_labels_noncompliance.values() if y=='mistrust'])

100%|██████████| 58361/58361 [00:17<00:00, 3307.70it/s]


patients labeled as    trustful: 54030
patients labeled as mistrustful: 484


In [7]:
# LABEL: whether patient got autopsy

autopsy_consent = []
autopsy_decline = []
for hadm_id,rows in tqdm.tqdm(notes.groupby('hadm_id')):
    consented = False
    declined = False
    for text in rows.text.values:
        for line in text.lower().split('\n'):
            if 'autopsy' in line:
                if 'decline' in line:
                    declined = True
                if 'not consent' in line:
                    declined = True
                if 'refuse' in line:
                    declined = True
                if 'denied' in line:
                    declined = True
                    
                if 'consent' in line:
                    consented = True
                if 'agree' in line:
                    consented = True
                if 'request' in line:
                    consented = True
                
    # probably some "declined donation but consented to autopsy" or something confusing. just ignore hard cases
    if consented and declined:
        continue

    if consented:
        autopsy_consent.append(hadm_id)
    if declined:
        autopsy_decline.append(hadm_id)
        
# binary labels
trust_labels_autopsy = {}
for hadm_id in autopsy_consent:
    trust_labels_autopsy[int(hadm_id)] = 'mistrust'
for hadm_id in autopsy_decline:
    trust_labels_autopsy[int(hadm_id)] = 'trust'
    
print 'patients labeled as    trustful:', len([y for y in trust_labels_autopsy.values() if y=='trust'   ])
print 'patients labeled as mistrustful:', len([y for y in trust_labels_autopsy.values() if y=='mistrust'])

100%|██████████| 58361/58361 [00:31<00:00, 1841.80it/s]


patients labeled as    trustful: 739
patients labeled as mistrustful: 270


<h1>Supervised Learning to Classify Trust-based Outcomes</h1>

In [8]:
# Useful ML functions

def data_split(ids, ratio=0.7):
    random.shuffle(ids)
    train = ids[:int(len(ids)*ratio) ]
    test  = ids[ int(len(ids)*ratio):]
    return train, test


def compute_stats(task, pred, P, ref, labels_map):
    scores = P[:,1] - P[:,0]
    res = compute_stats_binary(    task, pred, scores, ref, labels_map)
    return res



def compute_stats_binary(task, pred, P, ref, labels):
    # santiy check
    assert all(map(int,P>0) == pred)

    V = [0,1]
    n = len(V)
    assert n==2, 'sorry, must be exactly two labels (how else would we do AUC?)'
    conf = np.zeros((n,n), dtype='int32')
    for p,r in zip(pred,ref):
        conf[p][r] += 1

    print conf
    print
    
    tp = conf[1,1]
    tn = conf[0,0]
    fp = conf[1,0]
    fn = conf[0,1]

    precision   = tp / (tp + fp + 1e-9)
    recall      = tp / (tp + fn + 1e-9)
    sensitivity = tp / (tp + fn + 1e-9)
    specificity = tn / (tn + fp + 1e-9)

    f1 = (2*precision*recall) / (precision+recall+1e-9)

    accuracy = (tp+tn) / (tp+tn+fp+fn + 1e-9)
    
    print '\tspecificity %.3f' % specificity
    print '\tsensitivty: %.3f' % sensitivity

    # AUC
    if len(set(ref)) == 2:
        auc = sklearn.metrics.roc_auc_score(ref, P)
        print '\t\tauc:        %.3f' % auc

    print '\taccuracy:   %.3f' % accuracy
    print '\tprecision:  %.3f' % precision
    print '\trecall:     %.3f' % recall
    print '\tf1:         %.3f' % f1
    
    res = {'accuracy':accuracy, 'precision':precision, 'recall':recall, 'f1':f1, 
           'auc':auc, 'sensitivity':sensitivity, 'specificity':specificity}

    return res



def classification_results(svm, labels_map, X, Y, task):
    # for AUC
    P_ = svm.decision_function(X)

    # sklearn has stupid-ass changes in API when doing binary classification. make it conform to 3+
    if len(labels_map)==2:
        m = X.shape[0]
        P = np.zeros((m,2))
        P[:,0] = -P_
        P[:,1] =  P_
    else:
        P = P_

    train_pred = P.argmax(axis=1)

    # what is the predicted vocab without the dummy label?
    V = labels_map.keys()

    print task
    res = compute_stats(task, train_pred, P, Y, labels_map)
    print '\n'
    return res
    print

In [9]:
# Fit vectorizer for chartevents features
vect = DictVectorizer()
vect.fit(chartevents_features.values())
print 'num_features:', len(vect.get_feature_names())

num_features: 620


In [36]:
# display informative features

def classification_analyze(task, vect, clf, labels_map, num_feats=10):
    ind2feat =  { i:f for f,i in vect.vocabulary_.items() }

    labels = [label for label,i in sorted(labels_map.items(), key=lambda t:t[1])]
    coef_ = clf.coef_
    
    informative_feats = np.argsort(coef_)

    neg_features = informative_feats[0,:num_feats ]
    pos_features = informative_feats[0,-num_feats:]
    
    # display what each feature is
    print 'POS %s' % label
    for feat in reversed(pos_features):
        val = coef_[0,feat]
        word = ind2feat[feat]
        if val > 1e-4:
            print '\t%-25s: %7.4f' % (word,val)
        else:
            break
    print 'NEG %s' % label
    for feat in reversed(neg_features):
        val = coef_[0,feat]
        word = ind2feat[feat]
        if -val > 1e-4:
            print '\t%-25s: %7.4f' % (word,val)
        else:
            continue
    print '\n'

In [37]:
# vectorize task-specific labels
trust_Y_vect = {'mistrust': 1, 'trust': 0}
print trust_Y_vect

{'trust': 0, 'mistrust': 1}


In [38]:
# CLASSIFIER: noncompliance

noncompliance_cohort = list(set(trust_labels_noncompliance.keys()) & set(chartevents['hadm_id'].values))
print 'patients:', len(noncompliance_cohort)

# train/test split
noncompliance_train_ids, noncompliance_test_ids = data_split(noncompliance_cohort)

# select pre-computed features
noncompliance_train_features = [chartevents_features[hadm_id] for hadm_id in noncompliance_train_ids]
noncompliance_test_features  = [chartevents_features[hadm_id] for hadm_id in noncompliance_test_ids ]

# vectorize features
noncompliance_train_X = vect.transform(noncompliance_train_features)
noncompliance_test_X  = vect.transform(noncompliance_test_features)

# select labels
noncompliance_train_Y = [trust_Y_vect[trust_labels_noncompliance[hadm_id]] for hadm_id in noncompliance_train_ids]
noncompliance_test_Y  = [trust_Y_vect[trust_labels_noncompliance[hadm_id]] for hadm_id in noncompliance_test_ids ]

# fit model
noncompliance_svm = LogisticRegression(C=0.1, penalty='l1', tol=0.01)
noncompliance_svm.fit(noncompliance_train_X, noncompliance_train_Y)
print noncompliance_svm

# evaluate model
classification_results(noncompliance_svm, trust_Y_vect, noncompliance_train_X, noncompliance_train_Y, 'train: noncompliance')
classification_results(noncompliance_svm, trust_Y_vect,  noncompliance_test_X,  noncompliance_test_Y, 'test:  noncompliance')

# most informative features
classification_analyze('noncompliance', vect, noncompliance_svm, trust_Y_vect, num_feats=15)


patients: 54510
LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.01,
          verbose=0, warm_start=False)
train: noncompliance
[[37817   340]
 [    0     0]]

	specificity 1.000
	sensitivty: 0.000
		auc:        0.723
	accuracy:   0.991
	precision:  0.000
	recall:     0.000
	f1:         0.000


test:  noncompliance
[[16213   140]
 [    0     0]]

	specificity 1.000
	sensitivty: 0.000
		auc:        0.675
	accuracy:   0.991
	precision:  0.000
	recall:     0.000
	f1:         0.000


POS mistrust
	('riker-sas scale', 'agitated'):  0.6963
	('support systems', 'relative'):  0.2188
	('headache', 'not present'):  0.2151
	('riker-sas scale', 'sedated'):  0.1853
	('pain level', '0-none') :  0.1653
	('health care proxy', 'no'):  0.1567
	('education topic', 'medications'):  0.1424
	('social work consult', '1'):  0.1014
	('side r

In [39]:
# CLASSIFIER: autopsy

autopsy_cohort = list(set(trust_labels_autopsy.keys()) & set(chartevents['hadm_id'].values))
print 'patients:', len(autopsy_cohort)

# train/test split
autopsy_train_ids, autopsy_test_ids = data_split(autopsy_cohort)

# select pre-computed features
autopsy_train_features = [chartevents_features[hadm_id] for hadm_id in autopsy_train_ids]
autopsy_test_features  = [chartevents_features[hadm_id] for hadm_id in autopsy_test_ids ]

# vectorize features
autopsy_train_X = vect.transform(autopsy_train_features)
autopsy_test_X  = vect.transform(autopsy_test_features)

# select labels
autopsy_train_Y = [trust_Y_vect[trust_labels_autopsy[hadm_id]] for hadm_id in autopsy_train_ids]
autopsy_test_Y  = [trust_Y_vect[trust_labels_autopsy[hadm_id]] for hadm_id in autopsy_test_ids ]

# fit model
autopsy_svm = LogisticRegression(C=0.1, penalty='l1', tol=0.01)
autopsy_svm.fit(autopsy_train_X, autopsy_train_Y)
print autopsy_svm

# evaluate model
classification_results(autopsy_svm, trust_Y_vect, autopsy_train_X, autopsy_train_Y, 'train: autopsy')
classification_results(autopsy_svm, trust_Y_vect,  autopsy_test_X,  autopsy_test_Y, 'test:  autopsy')

# most informative features
classification_analyze('trust', vect, autopsy_svm, trust_Y_vect, num_feats=15)

patients: 997
LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.01,
          verbose=0, warm_start=False)
train: autopsy
[[515 182]
 [  0   0]]

	specificity 1.000
	sensitivty: 0.000
		auc:        0.627
	accuracy:   0.739
	precision:  0.000
	recall:     0.000
	f1:         0.000


test:  autopsy
[[216  84]
 [  0   0]]

	specificity 1.000
	sensitivty: 0.000
		auc:        0.530
	accuracy:   0.720
	precision:  0.000
	recall:     0.000
	f1:         0.000


POS mistrust
	('restraints evaluated', 'restraintreapply'):  0.0858
	('orientation', 'oriented x 3'):  0.0685
	('orientation', 'oriented x 2'):  0.0221
NEG mistrust
	('education readiness', 'no'): -0.0323
	('education method', 'explanation'): -0.0354
	('support systems', 'children'): -0.0438
	('safety measures', 'call light within reach'): -0.0451
	('education barrier', 

In [40]:
# Predict scores for all patients and save to file

# ordering of all chartevents features
chartevents_ids = set(chartevents['hadm_id'].values)
chartevents_X = vect.transform([chartevents_features[hadm_id] for hadm_id in chartevents_ids])

# Save ranking (i.e. confidence from trust classifier) of all patients on the noncompliance metric
with open('../data/mistrust_noncompliant.pkl', 'wb') as f:
    noncompliance_scores = dict(zip(chartevents_ids,noncompliance_svm.decision_function(chartevents_X)))
    pickle.dump(noncompliance_scores, f)
print 'DONE: noncompliance scores'

# Save ranking (i.e. confidence from trust classifier) of all patients on the autopsy metric
with open('../data/mistrust_autopsy.pkl', 'wb') as f:
    autopsy_scores = dict(zip(chartevents_ids,autopsy_svm.decision_function(chartevents_X)))
    pickle.dump(autopsy_scores, f)
print 'DONE: autopsy scores'    

DONE: noncompliance scores
DONE: autopsy scores


<h1>Sentiment Analysis as Mistrust Proxy</h1>

In [ ]:
# Sentiment Analysis
from pattern.en import sentiment
import os
import tqdm
import numpy as np

datadir = '/scratch/wboag/2018/iatrophobia/sequence/data/readable/all/'

sentiments = {}
#for hadm_id in tqdm.tqdm(bw_eol_cohort['hadm_id'].values):
for name in tqdm.tqdm(os.listdir(datadir)):
    #filename = '/scratch/wboag/2018/iatrophobia/sequence/data/readable/all/%s.txt' % hadm_id
    filename = os.path.join(datadir, name)
    hadm_id = int(name.split('.')[0])
    if not os.path.exists(filename):
        continue
        
    with open(filename, 'r') as f:
        text = f.read().split('================================================================================')[1]
        
    # Returns a (polarity, subjectivity)-tuple.
    sa = sentiment(text.split())
    #sa = sentiment(text)
    #print sa
    sentiments[hadm_id] = sa[0]
    #break

scores = np.array(sentiments.values())
mu = scores.mean()
std = scores.std()
sentiments = {k:-(v-mu)/std for k,v in sentiments.items()}
    
print 'sa:', len(sentiments)

with open('../data/neg_sentiment.pkl', 'wb') as f:
    pickle.dump(sentiments, f)
print 'DONE: negative sentiment scores'